In [9]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import os
import tiktoken
import re
import time
from tqdm import tqdm
import ast
import json

In [10]:
def chunk_data(data, size):
    return [data[i:i + size] for i in range(0, len(data), size)]

In [ ]:
def split_into_n_lists(full_list, n):
    # Determine the length of each sublist
    total_len = len(full_list)
    base_size = total_len // n
    remainder = total_len % n
    
    # Intialize lists
    lists = []
    start = 0
    
    # Create each sublist
    for i in range(n):
        # Add one more element to some of the lists to distribute the remainder
        end = start + base_size + (1 if i < remainder else 0)
        lists.append(full_list[start:end])
        start = end
        
    return lists

In [ ]:
def process_responses(responses):
    data_dict = {}

    # Define regex pattern to handle inconsistencies
    pattern = r"\(\s*'([^']+)'?\s*,\s*'([^']+\s?[^']*?)'?\s*,\s*(['\"]?[\d,\.]+['\"]?)\s*\)"

    # Process each stored response
    for response in responses:
        # Normalize the response to reduce issues with inconsistent formatting
        normalized_response = response.replace("\n", "").replace("), ", "),")
        matches = re.findall(pattern, normalized_response)
        for word1, word2, score in matches:
            # Concatenate words separated by whitespace in the second word
            word2 = word2.replace(" ", "")
            # Remove quotes if the score is captured as a quoted string
            if score.startswith(("'", '"')) and score.endswith(("'", '"')):
                score = score[1:-1]
            # Replace comma with dot in the score string to handle decimal numbers
            score = score.replace(",", ".")
            # Ensure no trailing non-numeric characters
            score = re.sub(r'[^\d.]', '', score)
            key = (word1, word2)
            try:
                # Attempt to convert score to float and add to dictionary
                if key not in data_dict:
                    data_dict[key] = []
                data_dict[key].append(float(score))
            except ValueError as e:
                # Log the error along with the problematic score for debugging
                print(f"Error converting '{score}' to float in the pair {key}: {e}")

    return data_dict

In [ ]:
def convert_dict_to_dataframe(data_dict):
    # Only include 

    # Create DataFrame columns based on the number of scores collected
    columns = ['word1', 'word2'] + [f'similarity_score_{i+1}' for i in range(len(next(iter(data_dict.values()))))]
    data_list = []
    
    # Populate data list for DataFrame creation
    for (word1, word2), scores in data_dict.items():
        data_list.append([word1, word2] + scores)
    
    # Create DataFrame
    df = pd.DataFrame(data_list, columns=columns)
    return df

In [11]:
def format_prompt(pairs, prompt):
    formatted_pairs = ', '.join([f"('{p[0]}', '{p[1]}')" for p in pairs])
    # return f"{prompt}\n\n---\n\n{formatted_pairs}"
    return f"{prompt} --- {formatted_pairs}"

In [12]:
def print_prompts(chunks, prompt):
    for chunk in chunks:
        print(format_prompt(chunk, prompt))

In [ ]:
def count_tokens_with_tiktoken(chunks, prompt):
    token_counts = []
    for chunk in chunks:
        formatted_prompt = format_prompt(chunk, prompt) # Format prompt
        tokens = encoding.encode(formatted_prompt) # Tokenize the formatted prompt
        token_counts.append(len(tokens)) # Count the tokens and add to the list
    
    return token_counts

In [ ]:
# def parse_content(content_str):
#     tuples = re.findall(r"\('(.*?)', '(.*?)', ([\d\.]+)\)", content_str)
#     return [(w1, w2, float(score)) for w1, w2, score in tuples]

In [ ]:
def get_responses(chunks, prompt, model, sample_size, delay):
    # Initialize list to store responses
    responses = [] 

    # Start timing the total processing
    start_time_total = time.time()

    # Set up the progress bar
    total_iterations = len(chunks) * sample_size
    pbar = tqdm(total=total_iterations, desc="Processing", unit="chunk")

    # Collect responses for each chunk and each sample
    for chunk in chunks:
        for _ in range(sample_size):
            # Format the prompts
            formatted_prompt = format_prompt(chunk, prompt)
            messages = [{"role": "user", "content": formatted_prompt}]

            # Call the OpenAI API
            completion = client.chat.completions.create(
                model=model,
                messages=messages,
                n=1)

            # Store the response content
            responses.append(completion.choices[0].message.content)

            # Update the progress bar after each sample
            pbar.update(1)

            # Delay after each API call
            time.sleep(delay)

    # Close the progress bar when done
    pbar.close()

    # Total processing time
    end_time_total = time.time()
    print(f"Total time taken: {end_time_total - start_time_total:.2f} seconds")

    return responses

In [ ]:
# def get_and_process_chat_completion(chunks, prompt, model, sample_size, delay):
#     # Initialize a dictionary to store data
#     word_dict = {}

#     # Start timing the total processing
#     start_time_total = time.time()

#     # Set up the progress bar
#     total_iterations = len(chunks) * sample_size
#     pbar = tqdm(total=total_iterations, desc="Processing", unit="chunk")

#     # Collect results for each chunk and each sample
#     for chunk in chunks:
#         for sample_index in range(sample_size):
#             # Format the prompts
#             formatted_prompt = format_prompt(chunk, prompt)
#             messages = [{"role": "user", "content": formatted_prompt}]

#             # Call the OpenAI API
#             completion = client.chat.completions.create(
#                 model=model,
#                 messages=messages,
#                 n=1)

#             # Extract content and convert into list
#             content = completion.choices[0].message.content
#             print(content) # Validate
#             list_of_tuples = ast.literal_eval(content)
#             # list_of_tuples = parse_content(completion.choices[0].message.content)

#             # Process word pairs and scores into dictionary
#             for w1, w2, score in list_of_tuples:
#                 if (w1, w2) not in word_dict:
#                     dict[(w1, w2)] = {'word1': w1, 'word2': w2}
#                 dict[(w1, w2)][f'similarity_score_{sample_index+1}'] = score

#             # Update the progress bar after each sample
#             pbar.update(1)

#             # Delay after each API call
#             time.sleep(delay)
            
#     # Close the progress bar when done
#     pbar.close()

#     # Total processing time
#     end_time_total = time.time()
#     print(f"Total time taken: {end_time_total - start_time_total:.2f} seconds")

#     return word_dict

In [ ]:
# def get_and_process_chat_completion(chunks, prompt, model, sample_size, delay):
#     # Initialize an empty Pandas DataFrame with word pairs
#     df = pd.DataFrame(columns=['word1', 'word2'])

#     # Start timing the total processing
#     start_time_total = time.time()

#     # Set up the progress bar
#     total_iterations = len(chunks) * sample_size
#     pbar = tqdm(total=total_iterations, desc="Processing", unit="chunk")

#     # Create columns for each sample
#     for sample_index in range(sample_size):
#         df[f'similarity_score_{sample_index+1}'] = None

#     # Collect results for each chunk and each sample
#     for chunk_index, chunk in enumerate(chunks):
#         for sample_index in range(sample_size):
#             # Format the prompts
#             formatted_prompt = format_prompt(chunk, prompt)
#             messages = [{"role": "user", "content": formatted_prompt}]

#             # Call the OpenAI API
#             completion = client.chat.completions.create(
#                 model=model,
#                 messages=messages,
#                 n=1)

#             # Extract content from completion
#             content = parse_content(completion.choices[0].message.content)
#             print(print)
            
#             # Process and add the scores to the DataFrame
#             for row_index, row in enumerate(content):
#                 if chunk_index == 0 and sample_index == 0:
#                     # For the first sample and first chunk, initialize the word pairs
#                     if row_index >= len(df):
#                         df = df.append({'word1': row[0], 'word2': row[1]}, ignore_index=True)
#                 df.loc[row_index, f'similarity_score_{sample_index+1}'] = row[2]
            
#             # Delay after each API call
#             time.sleep(delay)

#     # Close the progress bar when done
#     pbar.close()

#     # Total processing time
#     end_time_total = time.time()
#     print(f"Total time taken: {end_time_total - start_time_total:.2f} seconds")

#     return df

In [ ]:
def create_dataframe(data_dict):
    # Determine the maximum number of scores for any word pair
    max_scores = max(len(scores) for scores in data_dict.values())

    # Create DataFrame columns based on the maximum number of scores collected
    columns = ['word1', 'word2'] + [f'similarity_score_{i+1}' for i in range(max_scores)]
    data_list = []
    
    # Populate data list for DataFrame creation
    for (word1, word2), scores in data_dict.items():
        # Ensure each row has the same number of elements by filling missing scores with None
        full_scores = scores + [None] * (max_scores - len(scores))
        data_list.append([word1, word2] + full_scores)
    
    # Create DataFrame
    return pd.DataFrame(data_list, columns=columns)